## Create a frame for the blank level 0, KPF data
Size estimates from this link: https://exoplanets.atlassian.net/wiki/spaces/shrek/pages/2387738641/KPF+File+Size+Estimates

In [28]:
import numpy as np
import astropy
import astropy.io.fits as fits
from astropy.table import Table
import pandas as pd
import matplotlib.pyplot as plt

header_input = "KPF_Header_Keywords_L0_primary_v2.csv"
# Generate Primary header, using .csv generated from this google doc:
#     https://docs.google.com/spreadsheets/d/1KSnc97G3-a-KX-oOfFaUmeWDBvDIvkW247lQMb6KEsY/edit#gid=1802788034

In [29]:
# Setup dimensions of each green and red amplifier and create an HDU that will be the same 8x (2chips, 4 amps)
xdim = 2200
ydim = 2200
im = np.zeros([ydim,xdim],dtype=np.float32)

HDU_names = ['GREEN_AMP1','GREEN_AMP2','GREEN_AMP3','GREEN_AMP4',
             'RED_AMP1','RED_AMP2','RED_AMP3','RED_AMP4', 
             'CA_HK','EXPMETER','GUIDECAM']

# Header, read in .csv
head_amp = pd.read_csv(header_input,usecols= ['Keyword','Value','Description','Keep','HDU'])
head_amp = head_amp[ (head_amp['Keep'] == 'y') & (head_amp['HDU'] == '1')]
head_amp = head_amp[['Keyword','Value','Description']]
head_amp['Value'].fillna(0,inplace=True)

hdr_amp = fits.Header()
hdr_amp['XTENSION'] = 1

tuple_list_amp=tuple(head_amp.itertuples(index=False, name=None))

for tup in tuple_list_amp:
#    print(tup)
    hdr_amp.append(tup)
    
hdu1 = fits.ImageHDU(im,header = hdr_amp,name = HDU_names[0]) # 8 of these, four green, 
hdu2 = fits.ImageHDU(im,header = hdr_amp,name = HDU_names[1]) 
hdu3 = fits.ImageHDU(im,header = hdr_amp,name = HDU_names[2]) 
hdu4 = fits.ImageHDU(im,header = hdr_amp,name = HDU_names[3]) 

hdu5 = fits.ImageHDU(im,header = hdr_amp,name = HDU_names[4]) # 8 of these,  four red
hdu6 = fits.ImageHDU(im,header = hdr_amp,name = HDU_names[5]) 
hdu7 = fits.ImageHDU(im,header = hdr_amp,name = HDU_names[6]) 
hdu8 = fits.ImageHDU(im,header = hdr_amp,name = HDU_names[7]) 

hdu9 = fits.ImageHDU(im,name=HDU_names[8]) # CA_HK
hdu10 =fits.ImageHDU(im,name=HDU_names[9])
hdu11 = fits.ImageHDU(im,name=HDU_names[10])
print(repr(hdr_amp))

XTENSION=                    1                                                  
BSCALE  = '1       '           / nan                                            
BZERO   = '32768   '           / nan                                            
BUNIT   =                    0 / Image data units                               
AMPNAME = 'Amplifier 01'       / Amplifier name                                 
AMPNUM  = 'Amplifier 01'       / Amplifier number                               
AMPROW  = '0       '           / Amplifier row                                  
AMPCOL  = '0       '           / Amplifier column                               
AMPSEC  = '[1:1250,1:5000]'    / Amplifier section                              
DETSEC  = '[1:1152,1:4616]'    / Detector data section                          
DATASEC = '[28:1179,1:4616]'   / Image data section                             
BIASSEC1= '[1:27,1:5000]'      / Bias section 1                                 
BIASSEC2= '[28:1179,4617:500

In [30]:
# Header for all of the primary header, read in .csv
headP_small = pd.read_csv(header_input,usecols= ['Keyword','Value','Description','Keep','HDU'])
headP_small = headP_small[ (headP_small['Keep'] == 'y') & (headP_small['HDU'] == '0')]
headP_small=headP_small[['Keyword','Value','Description']]
headP_small['Value'].fillna(0,inplace=True)

hdr_primary = fits.Header()

# Create list of tuples to iterate over and append to header
tuple_list=tuple(headP_small.itertuples(index=False, name=None))

for tup in tuple_list:
#    print(tup)
    hdr_primary.append(tup)

empty_primary = fits.PrimaryHDU(header=hdr_primary) 
#print(repr(hdr_primary))

In [31]:
# FILE Names for simulated data and owncloud location:
sim_dir = '/Users/hisaacson/ownCloud/KPF-Pipeline-TestData/KPF-Pipeline-TestData/KPF_Simulated_Data/'

sim_sci_grn = 'KPF_simulated_L0_science.fits'
sim_sci_red = "science/kpf_red_L0_20210525T000000_quad.fits"

sim_hk = "HK/hk_sim_data.fits"

In [32]:
# Now insert the simulated data into the GREEN ccd HDUs
hdu_sim_grn = fits.open(sim_dir+sim_sci_grn)
hdu1.data = hdu_sim_grn['GREEN-AMP1'].data
hdu2.data = hdu_sim_grn['GREEN-AMP2'].data
hdu3.data = hdu_sim_grn['GREEN-AMP3'].data
hdu4.data = hdu_sim_grn['GREEN-AMP4'].data
#fullframe_grn = hdu_sim_grn['GREEN-CCD'].data # Looks empty

hdu_sim_grn.info()

#print(repr(hdu1.header))

#plt.figure(figsize=(20,10))
#plt.imshow(hdu1.data)

Filename: /Users/hisaacson/ownCloud/KPF-Pipeline-TestData/KPF-Pipeline-TestData/KPF_Simulated_Data/KPF_simulated_L0_science.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       7   ()      
  1  GREEN-AMP1    1 ImageHDU         8   (2200, 2200)   float32   
  2  GREEN-AMP2    1 ImageHDU         8   (2200, 2200)   float32   
  3  GREEN-AMP3    1 ImageHDU         8   (2200, 2200)   float32   
  4  GREEN-AMP4    1 ImageHDU         8   (2200, 2200)   float32   
  5  GREEN-CCD     1 ImageHDU         8   (4080, 4080)   float32   


In [33]:
# Now insert the simulated data into the RED ccd HDUs, file has FFI, but not used in L0 creation.
hdu_sim_red = fits.open(sim_dir+sim_sci_red)  

hdu5.data = hdu_sim_red['QUAD1'].data# data1_sim
hdu6.data = hdu_sim_red['QUAD2'].data
hdu7.data = hdu_sim_red['QUAD3'].data
hdu8.data = hdu_sim_red['QUAD4'].data

hdu_sim_red.info()

#plt.figure(figsize=(20,10))
#plt.imshow(hdu_sim)

Filename: /Users/hisaacson/ownCloud/KPF-Pipeline-TestData/KPF-Pipeline-TestData/KPF_Simulated_Data/science/kpf_red_L0_20210525T000000_quad.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       7   ()      
  1  QUAD1         1 ImageHDU         8   (2200, 2200)   float32   
  2  QUAD2         1 ImageHDU         8   (2200, 2200)   float32   
  3  QUAD3         1 ImageHDU         8   (2200, 2200)   float32   
  4  QUAD4         1 ImageHDU         8   (2200, 2200)   float32   
  5  FFI           1 ImageHDU         8   (4080, 4080)   float32   


In [34]:
#plt.figure(figsize=(20,10))
#plt.autoscale(enable=True,axis='both')
#plt.imshow(data2)
#plt.imshow(hdu4.data[:,0:],origin = 'lower',aspect = 'auto',vmin=0,interpolation = 'none')


In [35]:
# Fill the CA_HK HDU:
hdu_sim_hk = fits.open(sim_dir+sim_hk) # Currently float32. Should be int16?
hdu_sim_hk.info()

hdu9.data = hdu_sim_hk['PRIMARY'].data

#Get header keywords
head_hk = pd.read_csv(header_input,usecols= ['Keyword','Value','Description','Keep','HDU'])
head_hk = head_hk[ (head_hk['Keep'] == 'y') & (head_hk['HDU'] == '9')]
head_hk =head_hk[['Keyword','Value','Description']]
head_hk['Value'].fillna(0,inplace=True)

hdr_hk = fits.Header()

tuple_list_hk=tuple(head_hk.itertuples(index=False, name=None))

for tup in tuple_list_hk:
#    print(tup)
    hdr_hk.append(tup)

#plt.figure(figsize=(20,10))
#plt.imshow(hdu_sim_hk[0].data,interpolation = 'none',origin = 'lower', vmin=0, vmax=1100)
#print(hdu_sim_hk[0].data.max())
#hdu9.info()

Filename: /Users/hisaacson/ownCloud/KPF-Pipeline-TestData/KPF-Pipeline-TestData/KPF_Simulated_Data/HK/hk_sim_data.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       6   (1024, 255)   int16   


In [36]:
# Setup HDU10 , exposure meter [HDU-10].

dim_exp1 = 1024
dim_exp2 = 256
dim_exp3 = 600 # 10min, 1s cadence
im_expm = np.zeros([dim_exp1,dim_exp2,dim_exp3],dtype=np.int16)

hdu_expm = fits.ImageHDU(im_expm,name='EXPMETER') # Currently empty header    


In [37]:
# GUIDECAM (HDU10)

dim_g1 = 640
dim_g2 = 512

im_guider = np.zeros([dim_g1,dim_g2],dtype = np.int16)
hdu_gc = fits.ImageHDU(im_guider,name='GUIDECAM') # currently empty header.



In [38]:
# Create a BinTableHDU for the SOLAR_IRRADIANCE, HDU12
#arr = np.zeros(1000)
c1 = fits.Column(name='a', array=np.array([1, 20]), format='K')
hdu_solar = fits.BinTableHDU.from_columns([c1],name='SOLAR_IRRADIANCE')


In [39]:
hdul_all = fits.HDUList([empty_primary,hdu1,hdu2,hdu3,hdu4,hdu5,hdu6,hdu7,hdu8, hdu9,hdu_expm,hdu_gc,hdu_solar])
hdul_all.info()
hdul_all.writeto('L0_data_format_w_headers_20211001.fits',overwrite=True)
#print(hdual_all[0].head)

Filename: (No file associated with this HDUList)
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     112   ()      
  1  GREEN_AMP1    1 ImageHDU        23   (2200, 2200)   float32   
  2  GREEN_AMP2    1 ImageHDU        23   (2200, 2200)   float32   
  3  GREEN_AMP3    1 ImageHDU        23   (2200, 2200)   float32   
  4  GREEN_AMP4    1 ImageHDU        23   (2200, 2200)   float32   
  5  RED_AMP1      1 ImageHDU        23   (2200, 2200)   float32   
  6  RED_AMP2      1 ImageHDU        23   (2200, 2200)   float32   
  7  RED_AMP3      1 ImageHDU        23   (2200, 2200)   float32   
  8  RED_AMP4      1 ImageHDU        23   (2200, 2200)   float32   
  9  CA_HK         1 ImageHDU         8   (1024, 255)   int16   
 10  EXPMETER      1 ImageHDU         9   (600, 256, 1024)   int16   
 11  GUIDECAM      1 ImageHDU         8   (512, 640)   int16   
 12  SOLAR_IRRADIANCE    1 BinTableHDU     11   2R x 1C   ['K']   


In [40]:
# Open and check
output = fits.open('L0_data_format_w_headers_20211001.fits')
output.info()

Filename: L0_data_format_w_headers_20211001.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     112   ()      
  1  GREEN_AMP1    1 ImageHDU        23   (2200, 2200)   float32   
  2  GREEN_AMP2    1 ImageHDU        23   (2200, 2200)   float32   
  3  GREEN_AMP3    1 ImageHDU        23   (2200, 2200)   float32   
  4  GREEN_AMP4    1 ImageHDU        23   (2200, 2200)   float32   
  5  RED_AMP1      1 ImageHDU        23   (2200, 2200)   float32   
  6  RED_AMP2      1 ImageHDU        23   (2200, 2200)   float32   
  7  RED_AMP3      1 ImageHDU        23   (2200, 2200)   float32   
  8  RED_AMP4      1 ImageHDU        23   (2200, 2200)   float32   
  9  CA_HK         1 ImageHDU         8   (1024, 255)   int16   
 10  EXPMETER      1 ImageHDU         9   (600, 256, 1024)   int16   
 11  GUIDECAM      1 ImageHDU         8   (512, 640)   int16   
 12  SOLAR_IRRADIANCE    1 BinTableHDU     11   2R x 1C   [K]   


In [41]:
head=output[0].header
print(repr(head))

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
ORIGIN  = 'astropy.fits'''     / File Originator                                
INSTRUME= 'KPF     '           / Instrument Name                                
DATALVL = '0,1,2   '           / Data Product Base Level                        
DATATMP = 'T/F     '           / Data Temporary Intermediate Product            
OBSERVER= 'Classical Observer Name or Queue' / Observer Name                    
TIMESYS = 'UTC     '           / Time Scale                                     
PROGRAM = 'Keck Prog ID'       / Observing Program ID                           
PINAME  = 'Keck PI '           / Program PI Name                                
OBSTYPE = 'Sci'', ''Eng'',''

Check out this directory and simulated data to figure out what goes into the L0 file.
    ownCloud/KPF-Pipeline-TestData/KPF-Pipeline-TestData/KPF_Simulated_Data/science/
    
KPF_rays-1.0E+05_orders- 71-103_cal-lfc_20GHz_sci-solar_Planck_sky-solar_Planck_RV_30000ms_normalized_10018_Red.fits
KPF_rays-1.0E+05_orders- 71-103_cal-lfc_20GHz_sci-solar_Planck_sky-solar_Planck_RV_30000ms_normalized_10018_Red_quad.fits
KPF_rays-1.0E+05_orders-103-138_cal-lfc_20GHz_sci-solar_Planck_sky-solar_Planck_RV_30000ms_normalized_193_Green.fits
KPF_rays-1.0E+05_orders-103-138_cal-lfc_20GHz_sci-solar_Planck_sky-solar_Planck_RV_30000ms_normalized_193_Green_quad.fits
kpf_green_L0_20210517T000000.fits
kpf_green_wave.fits
kpf_red_L0_20210525T000000.fits
kpf_red_L0_20210525T000000_quad.fits
simulation_information.txt
    

In [42]:
#What is this data:
f_in = "science/KPF_rays-1.0E+05_orders- 71-103_cal-lfc_20GHz_sci-solar_Planck_sky-solar_Planck_RV_30000ms_normalized_10018_Red_quad.fits"
#f_in = "science/kpf_red_L0_20210525T000000_quad.fits"
test_data = fits.open(sim_dir+f_in)
test_data.info()

Filename: /Users/hisaacson/ownCloud/KPF-Pipeline-TestData/KPF-Pipeline-TestData/KPF_Simulated_Data/science/KPF_rays-1.0E+05_orders- 71-103_cal-lfc_20GHz_sci-solar_Planck_sky-solar_Planck_RV_30000ms_normalized_10018_Red_quad.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       7   ()      
  1  QUAD1         1 ImageHDU         8   (2200, 2200)   float32   
  2  QUAD2         1 ImageHDU         8   (2200, 2200)   float32   
  3  QUAD3         1 ImageHDU         8   (2200, 2200)   float32   
  4  QUAD4         1 ImageHDU         8   (2200, 2200)   float32   
  5  FFI           1 ImageHDU         8   (4080, 4080)   float32   


In [43]:
d="/Users/hisaacson/ownCloud/KPF-Pipeline-TestData/KPF-Pipeline-TestData/KPF_Simulated_Data/flat/"
f="KPF_rays-1.0E+05_orders-103-138_cal-incandescent_sci-incandescent_sky-incandescent_normalized_154_Green_quad.fits"
dat = fits.open(d+f)
dat.info()

Filename: /Users/hisaacson/ownCloud/KPF-Pipeline-TestData/KPF-Pipeline-TestData/KPF_Simulated_Data/flat/KPF_rays-1.0E+05_orders-103-138_cal-incandescent_sci-incandescent_sky-incandescent_normalized_154_Green_quad.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       7   ()      
  1  QUAD1         1 ImageHDU         8   (2200, 2200)   float32   
  2  QUAD2         1 ImageHDU         8   (2200, 2200)   float32   
  3  QUAD3         1 ImageHDU         8   (2200, 2200)   float32   
  4  QUAD4         1 ImageHDU         8   (2200, 2200)   float32   
  5  FFI           1 ImageHDU         8   (4080, 4080)   float32   
